In [65]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import requests
import ast

In [66]:
V_T_df=pd.read_csv('data/V_T.csv')


In [67]:
V=np.array(V_T_df).T


In [68]:
url='https://anilist.co/api/'
cid='selzla-6acux'
sec='eGi4fmsY9pV64E1fSTWJJ1'
username='lightningkyria'
params={'grant_type':"client_credentials",'client_id':cid,'client_secret':sec}
access=requests.post(url+'auth/access_token',data=params)
access_token=access.json()['access_token']

In [69]:
user_anime=requests.get(url+'user/'+username+'/animelist?access_token='+access_token)

In [70]:
completed = user_anime.json()['lists']['completed']

if 'watching' in user_anime.json()['lists']:
    watching = user_anime.json()['lists']['watching']
else:
    watching = []

if 'dropped' in user_anime.json()['lists']:
    dropped = user_anime.json()['lists']['dropped']
else:
    dropped = []

if 'on_hold' in user_anime.json()['lists']:
    on_hold = user_anime.json()['lists']['on_hold']
else:
    on_hold = []

if 'plan_to_watch' in user_anime.json()['lists']:
    plan_to_watch = user_anime.json()['lists']['plan_to_watch']
else:
    plan_to_watch = []

In [71]:
scores = []
for i in range(len(completed)):
    title=completed[i]['anime']['title_english'].encode("utf-8")
    anime_id=completed[i]['anime']['id']
    score=completed[i]['score_raw']
    scores.append([anime_id, score])

In [72]:
series_df=pd.read_csv('data/series_data.csv')

In [73]:
series_df['real_genres']=series_df['genres'].apply(lambda x: ast.literal_eval(x))

In [74]:
def isAdult(ID):
    if len(list(series_df[series_df['id']==ID]['adult'])):
        return list(series_df[series_df['id']==ID]['adult'])[0]
    else:
        return None

In [75]:
def hasGenre(ID, genre):
    if len(list(series_df[series_df['id']==ID]['real_genres']))>0:
        return genre in list(series_df[series_df['id']==ID]['real_genres'])[0]
    else:
        return None

In [76]:
def StartYear(ID):
    if len(list(series_df[series_df['id']==ID]['start_date']))>0:
        return int(list(series_df[series_df['id']==ID]['start_date'])[0][:4])
    else:
        return None

In [77]:
def numEpisodes(ID):
    if len(list(series_df[series_df['id']==ID]['total_episodes']))>0:
        return list(series_df[series_df['id']==ID]['total_episodes'])[0]
    else: 
        return None

In [78]:
def Type(ID):
    if len(list(series_df[series_df['id']==ID]['type']))>0:
        return list(series_df[series_df['id']==ID]['type'])[0]
    else:
        return None

In [79]:
Type(1)

'TV'

In [80]:
titles={}
for i in range(len(series_df)):
    ID = series_df['id'][i]
    title = series_df['title_english'][i]
    titles[ID] = title


In [81]:
user=np.array([row for row in scores if row[0] in np.array(series_df['id'])])
user=np.array([[row[0]-1, row[1]] for row in user])
# print user

In [82]:
# np.array([[row[0]-1, row[1]] for row in user])

In [83]:
user_vector = np.zeros(V.shape[1])
for row in user:
    user_vector[row[0]]=row[1]

In [84]:
new_user=user_vector[np.where(user_vector!=0)]
new_V=V.T[np.where(user_vector!=0)].T
s=np.linalg.lstsq(new_V.T,new_user.T)[0].T
new_R=np.dot(s,V)

In [85]:
seen = [row[0] for row in scores]
others = [watching, dropped, on_hold, plan_to_watch]
for x in others:
    for i in range(len(x)):
        seen.append(x[i]['anime']['id'])

In [86]:
rec=np.argsort(-1*new_R)
rec = rec+1
f_type = (True, 'TV')
f_genres = (False, 'Slice of Life')
f_year = (False, 2000, 2016)
f_episodes = (False, 1, 100)
f_adult = (False, False)
count = 0
for i in range(1000):
    if rec[i] not in seen:
        allowed = True
        if f_type[0]:
            if Type(rec[i])!=f_type[1]:
                allowed = False
        if f_genres[0]:
            if not hasGenre(rec[i], f_genres[1]):
                allowed = False
        if f_year[0]:
            if StartYear(rec[i])<f_year[1] or StartYear(rec[i])>f_year[2]:
                allowed = False
        if f_episodes[0]:
            if numEpisodes(rec[i])<f_episodes[1] or numEpisodes(rec[i])>f_episodes[2]:
                allowed = False 
        if f_adult[0]:
            if isAdult(rec[i])!=f_adult[1]:
                allowed = False
        if allowed:
            print rec[i], titles[rec[i]]
            count +=1 
    if count>15:
        break
    
    

18897 Nisekoi
20668 Monthly Girls' Nozaki-kun
20447 Noragami
20457 Black Bullet
20602 Amagi Brilliant Park
20462 Chaika -The Coffin Princess-
20722 Barakamon
20529 The Kawai Complex Guide to Manors and Hostel Behavior
20458 The Irregular at Magic High School
17265 Log Horizon
20031 D-Frag!
20483 Engaged to the Unidentified
17895 Golden Time
18153 Beyond the Boundary
20661 Terror in Resonance
17549 Non Non Biyori
